In [96]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [97]:
# import os
# import numpy as np
# from PIL import Image

# # Function to load and preprocess the images
# def load_images_from_folder(folder_path, target_size=(200, 200)):
#     images = []
#     for filename in os.listdir(folder_path):
#         img_path = os.path.join(folder_path, filename)
#         try:
#             # Open image and resize it
#             img = Image.open(img_path).convert('RGB')
#             img = img.resize(target_size)  # Resizing the image to a target size
#             img_array = np.array(img)  # Converting to NumPy array
#             images.append(img_array)
#         except Exception as e:
#             print(f"Failed to process {img_path}: {e}")
    
#     images = np.array(images)  # Convert list of images to a NumPy array
#     return images

# # Specify folder path and target image size
# folder_path = r"C:\Users\farel\Documents\Hology UB\data\HOLOGY 2024\processed"
# images_data = load_images_from_folder(folder_path)


In [98]:
import os
import numpy as np
from PIL import Image
import pandas as pd

# Function to load images using the 'id' from the CSV
def load_images_from_ids(folder_path, ids, target_size=(200, 200)):
    images = []
    for image_id in ids:
        img_filename = f"{image_id}.jpg"  # Assuming image filenames are of the format 'id.jpg'
        img_path = os.path.join(folder_path, img_filename)
        try:
            # Open image, resize, and convert to NumPy array
            img = Image.open(img_path).convert('RGB')
            img = img.resize(target_size)
            img_array = np.array(img)
            images.append(img_array)
        except Exception as e:
            print(f"Failed to load image {img_filename}: {e}")
    
    return np.array(images)

# Load the CSV file
csv_file_path = r"C:\Users\farel\Documents\Hology UB\data\train.csv"
labels_df = pd.read_csv(csv_file_path)

# Get the 'id' and 'label' columns from the CSV
image_ids = labels_df['id'].values  # The 'id' column from CSV
jenis_labels = labels_df['jenis'].values  # The 'label' column from CSV
warna_labels = labels_df['warna'].values  # The 'label' column from CSV

# Folder where images are stored
folder_path = r"C:\Users\farel\Documents\Hology UB\data\processed"

# Load images based on 'id' column
images_data = load_images_from_ids(folder_path, image_ids)
# images_data = images_data.astype('float32') / 255.0


In [99]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Step 1: Label encode the categorical labels into integers
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(jenis_labels)  # 'labels' should come from your CSV

# Step 2: Apply OneHotEncoder to the encoded labels
one_hot_encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = one_hot_encoder.fit_transform(encoded_labels.reshape(-1, 1))

label_encoder2 = LabelEncoder()
encoded_labels2 = label_encoder2.fit_transform(warna_labels)  # 'labels' should come from your CSV

# Step 2: Apply OneHotEncoder to the encoded labels
one_hot_encoder2 = OneHotEncoder(sparse_output=False)
one_hot_labels2 = one_hot_encoder2.fit_transform(encoded_labels2.reshape(-1, 1))


In [100]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Normalize the image data to [0, 1] range
images_data = images_data.astype('float32') / 255.0

# Split the data into training and testing sets
X_train, X_test, y_train1, y_test1, y_train2, y_test2 = train_test_split(
    images_data, one_hot_labels, one_hot_labels2, test_size=0.2, random_state=42)

# Convert the arrays to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train1 = tf.convert_to_tensor(y_train1)
y_train2 = tf.convert_to_tensor(y_train2)
y_test1 = tf.convert_to_tensor(y_test1)
y_test2 = tf.convert_to_tensor(y_test2)

from tensorflow.keras.utils import to_categorical




In [101]:
# Convert one-hot encoded binary labels to single binary values
if y_train1.ndim > 1:  # If y_train1 is one-hot encoded
    y_train1 = np.argmax(y_train1, axis=1)  # Convert to binary labels

if y_test1.ndim > 1:  # Same for test data
    y_test1 = np.argmax(y_test1, axis=1)

print("y_train1 shape after conversion:", y_train1.shape)  # Should be (batch_size,)
from tensorflow.keras.utils import to_categorical

# If y_train2 is not already one-hot encoded, one-hot encode it
if y_train2.ndim == 1:  # If y_train2 is class indices (shape: (batch_size,))
    y_train2 = to_categorical(y_train2, num_classes=5)  # Assuming 5 classes
    y_test2 = to_categorical(y_test2, num_classes=5)

print("y_train2 shape after encoding:", y_train2.shape)  # Should be (batch_size, num_classes)



y_train1 shape after conversion: (621,)
y_train2 shape after encoding: (621, 5)


In [127]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, Dropout, GlobalAveragePooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Input layer
input_layer = Input(shape=(200, 200, 3))

# Shared convolutional base
x = Conv2D(64, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(0.6)(x)  

# Output 1: Binary classification
output1 = Dense(1, activation='sigmoid', name='output1')(x)

# Output 2: Multi-class classification (5 classes)
output2 = Dense(5, activation='softmax', name='output2')(x)

# Define the model with two outputs
model = Model(inputs=input_layer, outputs=[output1, output2])

# Compile the model with accuracy as the metric
model.compile(optimizer='adam',
              loss={'output1': 'binary_crossentropy', 'output2': 'categorical_crossentropy'},
              metrics={'output1': 'accuracy', 'output2': 'accuracy'})

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.00001)



# Train the model with both outputs
history = model.fit(X_train, {'output1': y_train1, 'output2': y_train2},
                    validation_data=(X_test, {'output1': y_test1, 'output2': y_test2}),
                    epochs=10, batch_size=32, callbacks=[early_stopping])


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 829ms/step - loss: 4.7588 - output1_accuracy: 0.5532 - output1_loss: 1.4523 - output2_accuracy: 0.2991 - output2_loss: 3.1871 - val_loss: 1.6450 - val_output1_accuracy: 0.6410 - val_output1_loss: 0.6764 - val_output2_accuracy: 0.7692 - val_output2_loss: 0.7985
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 769ms/step - loss: 1.8915 - output1_accuracy: 0.5629 - output1_loss: 0.7904 - output2_accuracy: 0.6616 - output2_loss: 0.9206 - val_loss: 1.2458 - val_output1_accuracy: 0.6538 - val_output1_loss: 0.6030 - val_output2_accuracy: 0.8910 - val_output2_loss: 0.4558
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 754ms/step - loss: 1.5095 - output1_accuracy: 0.6469 - output1_loss: 0.7071 - output2_accuracy: 0.8079 - output2_loss: 0.6111 - val_loss: 1.0757 - val_output1_accuracy: 0.7821 - val_output1_loss: 0.5189 - val_output2_accuracy: 0.9231 - val_output2_loss: 0.3714
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 763ms/step - loss: 1.3208 - output1_accuracy: 0.6

In [128]:
# Get predictions from the model on the test data
predictions = model.predict(X_test)

# Split predictions for both outputs
pred_output1 = predictions[0]  # Binary classification predictions
pred_output2 = predictions[1]  # Multi-class classification predictions
# Convert binary predictions to 0 or 1
pred_output1_binary = np.round(pred_output1).astype(int)

# Convert multi-class predictions to class indices (e.g., choose the class with the highest probability)
pred_output2_classes = np.argmax(pred_output2, axis=1)

# Now compare with true labels (already converted earlier)
correct_binary = (pred_output1_binary.squeeze() == y_test1).astype(int)
correct_multiclass = (pred_output2_classes == np.argmax(y_test2, axis=1)).astype(int)

# Exact match: Both binary and multi-class predictions must be correct for each sample
exact_matches = (correct_binary & correct_multiclass)

# Calculate exact match ratio
exact_match_ratio = np.mean(exact_matches)

print(f"Exact Match Ratio: {exact_match_ratio:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 204ms/step
Exact Match Ratio: 0.8269


In [60]:
folder_path_test = r"C:\Users\farel\Documents\Hology UB\data\HOLOGY 2024\processed_test"
test_data = load_images_from_folder(folder_path_test)
test_data = test_data.astype('float32') / 255.0

# Make predictions on the test images
predictions = model.predict(test_data)

# If it's a classification model with softmax, get the class with the highest probability
predicted_classes = np.argmax(predictions, axis=1)

output_df = pd.DataFrame({
    'predictions': predicted_classes  # Only one column for the predictions
})

# Save to CSV (you can choose any filename)
output_csv_path = r"C:\Users\farel\Documents\Hology UB\data\HOLOGY 2024\test_predictions.csv"
output_df.to_csv(output_csv_path, index=False)

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step


In [61]:
submission = pd.read_csv('sample_submission.csv')

In [62]:
import os
import numpy as np
from PIL import Image
import pandas as pd

# Function to load images using filenames in the test folder
def load_test_images(folder_path, target_size=(200, 200)):
    images = []
    image_ids = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        try:
            # Open image and resize it to match the model input
            img = Image.open(img_path).convert('RGB')
            img = img.resize(target_size)
            img_array = np.array(img)
            images.append(img_array)

            # Extract image id (assuming filenames are 'id.jpg')
            image_id = os.path.splitext(filename)[0]
            image_ids.append(image_id)
        except Exception as e:
            print(f"Failed to load image {filename}: {e}")
    
    return np.array(images), image_ids

# Assuming your test folder path
test_folder = r"C:\Users\farel\Documents\Hology UB\data\HOLOGY 2024\processed_test"

# Load the test images and their corresponding IDs
test_images, test_image_ids = load_test_images(test_folder)

# Normalize the test images (assuming your model expects [0, 1] range)
test_images = test_images.astype('float32') / 255.0

# Make predictions using the trained model
predictions = model.predict(test_images)

# Assuming the model outputs probabilities for a multi-class classification task
predicted_classes = np.argmax(predictions, axis=1)

# Create a DataFrame with 'id' and 'predicted_class' columns
output_df = pd.DataFrame({
    'id': test_image_ids,
    'predicted_class': predicted_classes
})

# Save the DataFrame to a CSV file
output_csv_path = r"C:\Users\farel\Documents\Hology UB\data\HOLOGY 2024\test_predictions.csv"
output_df.to_csv(output_csv_path, index=False)

print(f"Predictions saved to {output_csv_path}")



11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step
Predictions saved to C:\Users\farel\Documents\Hology UB\data\HOLOGY 2024\test_predictions.csv


In [64]:
submission['id'] = submission['id'].astype(str)
output_df['id'] = output_df['id'].astype(str)


updated_submission_df = submission.merge(output_df[['id', 'predicted_class']], on='id', how='left')

# Step 3: Replace the 'label' column in submission_df with 'predicted_class'
updated_submission_df['jenis'] = updated_submission_df['predicted_class']

# Step 4: Drop the 'predicted_class' column as it was only for merging
updated_submission_df = updated_submission_df.drop(columns=['predicted_class'])

# Step 5: Save the updated submission file
updated_submission_path = r"C:\Users\farel\Documents\Hology UB\data\HOLOGY 2024\updated_submission.csv"
updated_submission_df.to_csv(updated_submission_path, index=False)
